In [ ]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
## need biopython to run: https://anaconda.org/anaconda/biopython
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
## need OligoArrayAux to run: http://www.unafold.org/Dinamelt/software/OligoArrayAux.php
import oligo_melting as OligoMelt

In [ ]:
sns.set()

## Featurization
### Functions
can definitely be improved upon

In [ ]:
def letter_at_position(string, letter, position):
    ## takes in a string, letter (both as strings), and position as an integer
    ## returns boolean whether the given letter is at the given position in the string
    return string[position:position+1].__eq__(letter)

def tuple_at_position(string, letters, position):
    ## takes in a string, 2 letters (as a single string) and position as an integer
    ## returns boolean whether the given letters are at the given position in the string
    return string[position:position+2].__eq__(letters)

def triple_at_position(string, letters, position):
    ## takes in a string, 3 letters (as a single string) and position as an integer
    ## returns boolean whether the given letters are at the given position in the string
    return string[position:position+3].__eq__(letters)

def find_PAM(sequence, PAM):
    ## takes in sequence and PAM as strings representing genetic target sequence and the promotospacer-adjacent motif
    ## of the gene editing complex
    ## returns the position as an int or -1 if not found
    match = re.search(PAM, sequence)
    return match.start()

def gc_count(string):
    ## takes in a string
    ## returns an integer representing the number of G or C letters in that string
    count = 0
    for letter in string:
        if letter.__eq__('G') | letter.__eq__('C'):
            count=count+1
    return count

## Doench et al 2014 DataFrame

In [ ]:
df_doench=pd.ExcelFile("Doench 2014 Supplemental Tables.xlsx")
print(df_doench.sheet_names)

In [ ]:
df_table_features=pd.read_excel("Doench 2014 Supplemental Tables.xlsx", sheet_name="Supp Table 8")
df_table_features.head()

In [ ]:
df_table_data=pd.read_excel("Doench 2014 Supplemental Tables.xlsx", sheet_name="Supp Table 10")
df_table_data.head()

Need to find more data

### Isolating necessary data into one dataframe
- combine only necessary columns into one succinct dataframe
    - include anything that could be useful to models
- plot some graphs to visualize data

In [ ]:
## to do
df_features=df_table_sgRNA_scores[["Sequence","Expanded Sequence","sgRNA Score"]]
df_features["sgRNA Rank"] = df_features["sgRNA Score"].rank(method='max')
n = df_features["sgRNA Rank"].size
# D=(r − 3/8) / (n + 1/4)
df_features["sgRNA Normalized"] = (df_features["sgRNA Rank"] - (3/8)) / (n + 1/4)
df_features.head()

#### Normalization Verification

In [ ]:
plt.hist(df_V2["sgRNA Normalized"], alpha=0.5, density=True, label='Normalized sgRNA scores')
plt.hist(df_V2["sgRNA Score"], alpha=0.5, density=True, label='Raw sgRNA scores')
plt.ylabel('Probability density')
plt.title("V2 Dataset Histogram sgRNA scores")
plt.legend()

### V1 Featurization
can definitely be improved upon

In [ ]:
df_V1 = df_features.copy(deep=True) ## creates a hard copy of supplementary table 7 data to work with
df_V1['Sequence'] = df_V1['Sequence'].astype('string') ## sets data of sequence to String for parsing
df_V1['Expanded Sequence'] = df_V1['Expanded Sequence'].astype('string') ## sets data of sequence to String for parsing
df_V1['Expanded Sequence']

In [ ]:
V1_bool_parameters=["GC<10","GC>10"]

for k in range(30):
    ## creates columns for each nucleotide at each position in sequence
    for x in ['A',"G","C","T"]:
        df_V1[x + str(k+1)] = letter_at_position(df_V1['Sequence'],x,k)
        V1_bool_parameters.append(x + str(k+1))
        if k < 29:
            for y in ["A","G","C","T"]:
                df_V1[x + y + str(k+1)] = tuple_at_position(df_V1['Sequence'],x+y,k)
                V1_bool_parameters.append(x + y + str(k+1))
        else:
            for y in ["A","G","C","T"]:
                df_V1[x + y + "?"] =  False # not technically correct but will work as placeholder
                df_V1[x + y + "_PAM"] = False # this as well
                V1_bool_parameters.append(x + y + "?")
                V1_bool_parameters.append(x + y + "_PAM")
        df_V1[x + "?"] = False # same with this
        V1_bool_parameters.append(x + "?")
        
df_V1['GC Count'] = -1 ## creates column for GC count
df_V1["PAM position (expanded sequence)"] = -1

df_V1['Melting Temp Wallace'] = 0
df_V1['Melting Temp GC'] = 0
df_V1['Melting Temp NN1'] = 0
df_V1['Melting Temp NN2'] = 0
df_V1['Melting Temp NN3'] = 0
df_V1['Melting Temp NN4'] = 0

df_V1["GC<10"] = False
df_V1["GC>10"] = False

In [ ]:
for a in range(df_V1['Sequence'].size):
    ## loops through each sequence and computes the necessary one-hot nucleotide encoding features
    for k in range(30):
        for x in ['A',"G","C","T"]:
            df_V1[x + str(k+1)][a] = letter_at_position(df_V1['Sequence'][a],x,k)
            if k < 29:
                for y in ["A","G","C","T"]:
                    df_V1[x + y + str(k+1)][a] = tuple_at_position(df_V1['Sequence'][a],x+y,k)
                    if (not df_V1[x + y + "?"][a]):
                        df_V1[x + y + "?"][a] = df_V1[x + y + str(k+1)][a]
            
            if (not df_V1[x + "?"][a]):
                df_V1[x + "?"][a] = df_V1[x + str(k+1)][a]
                        
    ## generates the GC count data for each column
    df_V1['GC Count'][a] = gc_count(df_V1['Sequence'][a])
    ## calculates the PAM position for each sequence
    df_V1["PAM position (expanded sequence)"][a] = find_PAM(df_V1["Expanded Sequence"][a],"GG")

In [ ]:
for a in range(df_V1['Expanded Sequence'].size):
    df_V1["PAM position (expanded sequence)"][a] = find_PAM(df_V1["Expanded Sequence"][a],"GG")
    for x in ['A',"G","C","T"]:
        for y in ['A',"G","C","T"]:
            df_V1[x + y + "_PAM"] = tuple_at_position(df_V1["Expanded Sequence"][a],
                                                      x+y,
                                                      df_V1["PAM position (expanded sequence)"][a]-3)
    seq = Seq(df_V1["Expanded Sequence"][a])
    df_V1["Melting Temp Wallace"][a] = mt.Tm_Wallace(seq)
    df_V1["Melting Temp GC"][a] = mt.Tm_GC(seq)
    df_V1["Melting Temp NN1"][a] = mt.Tm_NN(seq, nn_table=mt.DNA_NN1)
    df_V1["Melting Temp NN2"][a] = mt.Tm_NN(seq, nn_table=mt.DNA_NN2)
    df_V1["Melting Temp NN3"][a] = mt.Tm_NN(seq, nn_table=mt.DNA_NN3)
    df_V1["Melting Temp NN4"][a] = mt.Tm_NN(seq, nn_table=mt.DNA_NN4)
    
    df_V1["GC<10"][a] = df_V1["GC Count"][a] < 10
    df_V1["GC>10"][a] = df_V1["GC Count"][a] > 10

In [ ]:
for v1 in V1_bool_parameters:
    df_V1[v1]=df_V1[v1].astype(int)

In [ ]:
df_V1.to_csv('Doench2014_V1_Featurized.csv')
df_V1.head()

### V2 Featurization
can definitely be improved upon

In [ ]:
df_V2 = df_V1.copy(deep=True)
V2_bool_parameters=V1_bool_parameters

In [ ]:
df_V2["AAAAA"] = "AAAAA" in df_V2["Expanded Sequence"] ## initializes each column to false
df_V2["GGGGG"] = "GGGGG" in df_V2["Expanded Sequence"]
df_V2["CCCCC"] = "CCCCC" in df_V2["Expanded Sequence"]
df_V2["TTTTT"] = "TTTTT" in df_V2["Expanded Sequence"]
V2_bool_parameters.append(["AAAAA","GGGGG","CCCCC","TTTTT"])
df_V2.head()

In [ ]:
for k in range(29):
    for x in ['A',"G","C","T"]:
        for y in ['A',"G","C","T"]:
            for z in ["A","G","C","T"]:
                
                if k < 28:
                    df_V2[x + y + z + str(k+1)] = False
                    V2_bool_parameters.append(x + y + z + str(k+1))
                else:
                    df_V2[x+y+z] = False
                    V2_bool_parameters.append(x+y+z)
df_V2["Delta G"] = 0
df_V2.head()

In [ ]:
for a in range(df_V2['Expanded Sequence'].size):
    df_V2["AAAAA"][a] = "AAAAA" in df_V2["Expanded Sequence"][a]
    df_V2["GGGGG"][a] = "GGGGG" in df_V2["Expanded Sequence"][a]
    df_V2["CCCCC"][a] = "CCCCC" in df_V2["Expanded Sequence"][a]
    df_V2["TTTTT"][a] = "TTTTT" in df_V2["Expanded Sequence"][a]
    
    for k in range(29):
        for x in ['A',"G","C","T"]:
            for y in ['A',"G","C","T"]:
                for z in ["A","G","C","T"]:

                    if k < 28:
                        df_V2[x + y + z + str(k+1)][a] = triple_at_position(df_V2["Expanded Sequence"][a], x+y+z, k)
                        if (not df_V2[x+y+z][a]):
                            df_V2[x+y+z][a] = df_V2[x + y + z + str(k+1)][a]

In [ ]:
for a in range(df_V2['Sequence'].size):
    ## note that this uses the 20-n.t. guide RNA sequence
    ## all previously calculated features use the 30-oligo target sequence
    
    # # Calculate melting temperature for 25uM oligos
    # (name, g, h, s, tm, seq)
    df_V2["Delta G"][a] = OligoMelt.Duplex.calc_tm(df_V2["Sequence"][a])[1]

In [ ]:
for v2 in V2_bool_parameters:
    df_V2[v2]=df_V2[v2].astype(int)

In [ ]:
df_V2.to_csv('Doench2014_V2_Featurized.csv')
df_V2.head()

### V3

In [ ]:
# to do